In [1]:
import torch
from torch import nn

In [2]:
def create_board():
    us = torch.zeros(3, 3)
    them = torch.zeros(3, 3)
    return torch.stack([us, them])

In [3]:
state = create_board()

In [4]:
state

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [5]:
def get_move_coords(move_index):
    return move_index % 3, move_index // 3

def is_valid_move(state, move_index):
    if move_index < 0 or move_index >= 9:
        return False
    x, y = get_move_coords(move_index)
    return torch.all(state[:, x, y] == 0).item()

In [6]:
is_valid_move(state, 6)

True

In [9]:
def make_move(state, move_index):
    assert(is_valid_move(state, move_index))
    x, y = get_move_coords(move_index)
    us, them = state.unbind()
    us = us.clone()
    us[x, y] = 1
    return torch.stack([them, us])

In [10]:
next_state = make_move(state, 4)

In [11]:
next_state

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 1., 0.],
         [0., 0., 0.]]])

In [12]:
state

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])